In [2]:
#q8
!hdfs dfsadmin -D 'fs.defaultFS=hdfs://boss:9000' -report -live

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 6616768512 (6.16 GB)
DFS Remaining: 6085660672 (5.67 GB)
DFS Used: 531107840 (506.50 MB)
DFS Used%: 8.03%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.3:9866 (p4-dn-2.p4_default)
Hostname: 0355590adf2f
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 267231232 (254.85 MB)
Non DFS Used: 22494244864 (20.95 GB)
DFS Remaining: 3042799616 (2.83 GB)
DFS Used%: 1.03%
DFS Remaining%: 11.78%
Configured Cache Capacit

In [ ]:
# for autograder
import time
time.sleep(30)

In [19]:
import requests

status = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
status = status.json()
print(status)


200
{'FileStatus': {'accessTime': 1701036627785, 'blockSize': 1048576, 'childrenNum': 0, 'fileId': 16386, 'group': 'supergroup', 'length': 174944099, 'modificationTime': 1701036633888, 'owner': 'root', 'pathSuffix': '', 'permission': '644', 'replication': 1, 'storagePolicy': 0, 'type': 'FILE'}}


In [25]:
block_size = status['FileStatus']['blockSize']
length = status['FileStatus']['length']
locs = {'lost':0}
alive = []
for i in range(0, length, block_size):
    r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&noredirect=true&offset="+str(i))
    if r.status_code == 403:
        locs['lost'] += 1
        continue
    full_url = r.json()['Location']
    url = full_url.split(':')[1][2:]
    alive.append(i)
    if url in locs:
        locs[url] += 1
    else:
        locs[url] = 1
#q9
print(locs)

{'lost': 82, '0355590adf2f': 85}


In [26]:
import pyarrow as pa
import pyarrow.fs
import io
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
f = hdfs.open_input_file("/single.csv")

In [32]:
skip_next = False
count = 0
string = b'Single Family'
for i in alive:
    line = f.read_at(block_size,i)
    # line = line.split("\n")
    # if skip_next:
    #     # if already counted the string in the earlier part of the line, skip this next one
    #     skip_next = False
    #     line = line[1:]
    if string in line:
        count += 1

In [34]:
#q10
print(count)

85
